<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Thank you for taking the time to improve the project! It is now accepted. Good luck on the next sprint!

</div>

**Review**

Hi, my name is Dmitry and I will be reviewing your project.
  
You can find my comments in colored markdown cells:
  
<div class="alert alert-success">
  If everything is done successfully.
</div>
  
<div class="alert alert-warning">
  If I have some (optional) suggestions, or questions to think about, or general comments.
</div>
  
<div class="alert alert-danger">
  If a section requires some corrections. Work can't be accepted with red comments.
</div>
  
Please don't remove my comments, as it will make further review iterations much harder for me.
  
Feel free to reply to my comments or ask questions using the following template:
  
<div class="alert alert-info">
  For your comments and questions.
</div>
  
First of all, thank you for turning in the project! You did a pretty good job overall, but there are a couple of problems that need to be fixed before the project is accepted. Let me know if you have questions!

# Project Title: Customer Churn Prediction for Beta Bank

# Project Description

Beta Bank, a prominent financial institution, has been facing a growing issue - customers leaving the bank over time. As a cost-effective strategy, Beta Bank has decided to prioritize retaining existing customers over acquiring new ones. In order to achieve this, the bank aims to predict which customers are likely to leave soon. To tackle this challenge, we have access to historical data on customer behavior and contract terminations.

Project Objectives:

Predict Customer Churn: Build a machine learning model that predicts whether a customer is likely to leave Beta Bank in the near future.

Maximize F1 Score: Develop a model with the highest possible F1 score, aiming for a minimum F1 score of 0.59.

AUC-ROC Evaluation: Measure the AUC-ROC (Area Under the Receiver Operating Characteristic) metric to assess the model's performance and compare it with the F1 score.

Project Steps:

Data Preparation: Download and preprocess the dataset, handling missing values, encoding categorical features, and scaling numeric attributes.

Class Imbalance Analysis: Examine the balance of classes in the dataset and understand the severity of the class imbalance.

Model Training without Balancing: Train initial models without addressing class imbalance and evaluate their performance.

Improve Model Quality: Apply at least two techniques to fix class imbalance, such as upsampling, downsampling, or using class weights. Select the best model parameters through hyperparameter tuning.

Model Training and Validation: Train different models on training and validation sets, comparing their F1 scores and AUC-ROC values.

Select the Best Model: Choose the model with the highest F1 score as the best-performing model.

Final Testing: Evaluate the selected model on the test set and report the F1 score and AUC-ROC score.

By executing these steps, Beta Bank aims to identify customers at risk of churning and take proactive measures to retain them, ultimately improving customer retention and reducing costs associated with customer acquisition.






In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold


In [2]:
# Step 1: Data Preparation
data = pd.read_csv('/datasets/Churn.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


<div class="alert alert-success">
<b>Reviewer's comment</b>

The data was loaded and inspected

</div>

In [3]:
#Based on loose correlations above, fill NaNs with the mean of grouped columns with highest degree of correlation.
data['Tenure'] = data['Tenure'].fillna(data.groupby(['IsActiveMember', 'HasCrCard', 'NumOfProducts'])['Tenure'].transform('mean').round(0))
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


<div class="alert alert-success">
<b>Reviewer's comment</b>

Missing values were dealt with reasonably

</div>

In [4]:
# Encode categorical features
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data = pd.get_dummies(data, columns=['Geography'], drop_first=True)
data

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,0,42,2.0,0.00,1,1,1,101348.88,1,0,0
1,2,15647311,Hill,608,0,41,1.0,83807.86,1,0,1,112542.58,0,0,1
2,3,15619304,Onio,502,0,42,8.0,159660.80,3,1,0,113931.57,1,0,0
3,4,15701354,Boni,699,0,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,5,15737888,Mitchell,850,0,43,2.0,125510.82,1,1,1,79084.10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,1,39,5.0,0.00,2,1,0,96270.64,0,0,0
9996,9997,15569892,Johnstone,516,1,35,10.0,57369.61,1,1,1,101699.77,0,0,0
9997,9998,15584532,Liu,709,0,36,7.0,0.00,1,0,1,42085.58,1,0,0
9998,9999,15682355,Sabbatini,772,1,42,3.0,75075.31,2,1,0,92888.52,1,1,0


<div class="alert alert-success">
<b>Reviewer's comment</b>

Categorical features were encoded

</div>

In [5]:
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2.0,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1.0,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8.0,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2.0,125510.82,1,1,1,79084.10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5.0,0.00,2,1,0,96270.64,0,0,0
9996,516,1,35,10.0,57369.61,1,1,1,101699.77,0,0,0
9997,709,0,36,7.0,0.00,1,0,1,42085.58,1,0,0
9998,772,1,42,3.0,75075.31,2,1,0,92888.52,1,1,0


<div class="alert alert-success">
<b>Reviewer's comment</b>

Makes sense to drop these columns

</div>

In [6]:
# Splitting the data into features (X) and the target variable (y)
X = data.drop('Exited', axis=1)
y = data['Exited']

In [7]:
# Step 2: Class Imbalance Examination
class_balance = y.value_counts()
print("Class Balance:")
print(class_balance)


Class Balance:
0    7963
1    2037
Name: Exited, dtype: int64


<div class="alert alert-success">
<b>Reviewer's comment</b>

Class imbalance was noted

</div>

In [8]:
# Step 3: Model Training without Balancing

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Train the model on the training set
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Evaluate the model on the validation set
y_pred_rf_val = rf_classifier.predict(X_val)
f1_rf_val = f1_score(y_val, y_pred_rf_val)
roc_auc_rf_val = roc_auc_score(y_val, rf_classifier.predict_proba(X_val)[:, 1])

print("Random Forest Classifier (Validation Set):")
print(f"F1 Score: {f1_rf_val}")
print(f"AUC-ROC Score: {roc_auc_rf_val}")


Random Forest Classifier (Validation Set):
F1 Score: 0.5764966740576497
AUC-ROC Score: 0.853134845255745


<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

Only the final model should be evaluated on the test set. Before that please use either a separate validation set or cross-validation to evaluate the models. The idea is that the test set is used to get an unbiased estimate of the final model's generalization performance and that is only possible if it is not used to make any decisions about the models.

</div>

<div class="alert alert-info">
  Corrected
</div>

<div class="alert alert-danger">
<s><b>Reviewer's comment V2</b>

The final model is the single *final* model, the one you select to put in production. It makes no sense to call each model you evaluate a final model :)

</div>

<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Great!

</div>

In [9]:
from sklearn.utils import shuffle

def upsample(x_train, y_train, repeat):
    x_zeros = x_train[y_train == 0]
    x_ones = x_train[y_train == 1]
    y_zeros = y_train[y_train == 0]
    y_ones = y_train[y_train == 1]

    features_upsampled = pd.concat([x_zeros] + [x_ones] * repeat)
    target_upsampled = pd.concat([y_zeros] + [y_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled

# Upsample the minority class in the training set (e.g., repeat=4)
X_train_upsampled, y_train_upsampled = upsample(X_train, y_train, 4)

print("Upsampled Data:")
print("X_train_upsampled shape:", X_train_upsampled.shape)
print("y_train_upsampled shape:", y_train_upsampled.shape)


Upsampled Data:
X_train_upsampled shape: (11359, 11)
y_train_upsampled shape: (11359,)


<div class="alert alert-success">
<b>Reviewer's comment</b>

Upsampling was applied correctly

</div>

In [10]:
def downsample(x_train, y_train, fraction):
    x_zeros = x_train[y_train == 0]
    x_ones = x_train[y_train == 1]
    y_zeros = y_train[y_train == 0]
    y_ones = y_train[y_train == 1]
    
    features_downsampled = pd.concat(
        [x_zeros.sample(frac=fraction, random_state=12345)]
        + [x_ones]
    )
    target_downsampled = pd.concat(
        [y_zeros.sample(frac=fraction, random_state=12345)]
        + [y_ones]
    )

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345
    )

    return features_downsampled, target_downsampled

# Downsample the majority class in the training set (e.g., fraction=0.25)
X_train_downsampled, y_train_downsampled = downsample(X_train, y_train, 0.25)

print("Downsampled Data:")
print("X_train_downsampled shape:", X_train_downsampled.shape)
print("y_train_downsampled shape:", y_train_downsampled.shape)

Downsampled Data:
X_train_downsampled shape: (2840, 11)
y_train_downsampled shape: (2840,)


HEY, I was trying to run the last three codes and it does not seems to work. I am not sure what to do. Could you please let me know how to fix it and what should I add more?

In [11]:
3*3*3*3

81

<div class="alert alert-warning">
<b>Reviewer's comment</b>

Not sure what you mean by 'does not seem to work'. Did it give you an error? What kind? In any case please direct questions like this to a tutor. The project needs to be fully working without errors to be ready for review :)
    
My best guess for what you meant is that the cell below takes a long time to run: that is because it needs to train `(3*3*3*3)*5 = 405` models (81 combinations of hyperparameters times 5, as you're doing 5-fold cross-validation). You can try a smaller number of combinations of hyperparameters

</div>

In [12]:
# Split the data into training and validation sets (use a smaller portion for validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the parameter grid for hyperparameter tuning
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize a RandomForestClassifier with balanced class weights
rf_classifier_class_weight = RandomForestClassifier(random_state=42, class_weight='balanced')

# Perform hyperparameter tuning using GridSearchCV
grid_search_rf_class_weight = GridSearchCV(rf_classifier_class_weight, param_grid_rf, scoring='f1', cv=3)
grid_search_rf_class_weight.fit(X_train, y_train)

# Get the best estimator and fit it on the entire training set
best_rf_classifier_class_weight = grid_search_rf_class_weight.best_estimator_
best_rf_classifier_class_weight.fit(X_train, y_train)

# Predict on the validation set and evaluate
y_pred_rf_class_weight_val = best_rf_classifier_class_weight.predict(X_val)
f1_rf_class_weight_val = f1_score(y_val, y_pred_rf_class_weight_val)
roc_auc_rf_class_weight_val = roc_auc_score(y_val, best_rf_classifier_class_weight.predict_proba(X_val)[:, 1])

print("Random Forest Classifier with Class Weights (Validation Set):")
print(f"F1 Score: {f1_rf_class_weight_val}")
print(f"AUC-ROC Score: {roc_auc_rf_class_weight_val}")



Random Forest Classifier with Class Weights (Validation Set):
F1 Score: 0.5929648241206029
AUC-ROC Score: 0.8386713092189195


In [13]:
# Initialize a RandomForestClassifier without class weights
rf_classifier_upsampled = RandomForestClassifier(random_state=42)

# Perform hyperparameter tuning using GridSearchCV
grid_search_rf_upsampled = GridSearchCV(rf_classifier_upsampled, param_grid_rf, scoring='f1', cv=3)
grid_search_rf_upsampled.fit(X_train, y_train)

# Get the best estimator and fit it on the entire training set
best_rf_classifier_upsampled = grid_search_rf_upsampled.best_estimator_
best_rf_classifier_upsampled.fit(X_train, y_train)

# Predict on the validation set and evaluate
y_pred_rf_upsampled_val = best_rf_classifier_upsampled.predict(X_val)
f1_rf_upsampled_val = f1_score(y_val, y_pred_rf_upsampled_val)
roc_auc_rf_upsampled_val = roc_auc_score(y_val, best_rf_classifier_upsampled.predict_proba(X_val)[:, 1])

print("\nRandom Forest Classifier without Upsampling (Validation Set):")
print(f"F1 Score: {f1_rf_upsampled_val}")
print(f"AUC-ROC Score: {roc_auc_rf_upsampled_val}")



Random Forest Classifier without Upsampling (Validation Set):
F1 Score: 0.5495495495495495
AUC-ROC Score: 0.8307306904037932


<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

1. Using an upsampled dataset for cross-validation is not appropriate: we can end up with identical examples in train and validation subsets in some folds, inflating the score. For cross-validation we need to apply upsampling in each fold separtely, and only upsample the whole train set just before retraining the final model before evaluating it on the test set (if we choose upsampling as the best method.
    
2. Please try using class weights and upsampling separately, so that we could compare the effect of the two techniques

</div>

<div class="alert alert-danger">
<S><b>Reviewer's comment V2</b>

Not sure what the meaning of this loop is:
    
```python
# Perform cross-validation with class weights and without upsampling
for train_idx, val_idx in cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Define and train the Random Forest Classifier with class weights on the fold
    rf_classifier_class_weight = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'),
                                              param_grid_rf, scoring='f1', cv=3)
    rf_classifier_class_weight.fit(X_train_fold, y_train_fold)
    
    # Store the best estimator for class weights
    best_estimators_rf_class_weight.append(rf_classifier_class_weight.best_estimator_)

    # Define and train the Random Forest Classifier on the fold (without class weights or upsampling)
    rf_classifier_upsampled = GridSearchCV(RandomForestClassifier(random_state=42),
                                           param_grid_rf, scoring='f1', cv=3)
    rf_classifier_upsampled.fit(X_train_fold, y_train_fold)
    
    # Store the best estimator for upsampling
    best_estimators_rf_upsampled.append(rf_classifier_upsampled.best_estimator_)
```

Remember that GridSearchCV is doing cross-validation under the hood, so you're fitting it on 3 parts of the train set (which is not what cross-validation is: in cross-validation you fit the data using the train fold and evaluate it using the validation fold, then the scores in different folds can be averaged to produce a single number) and putting the best estimators from the grid searches in a list, which is never used for anything: after this loop you just train the model with the default hyperparameters.
    
Another thing is that the `rf_classifier_upsampled` is not actually trained on upsampled data
    
</div>

<div class="alert alert-info">
  Corrected
</div>

<div class="alert alert-success">
<b>Reviewer's comment V3</b>

For a better fit of upsampling and cross-validation, check out imblearn library. In particular you can use an [imblearn pipeline](https://imbalanced-learn.org/stable/references/generated/imblearn.pipeline.Pipeline.html#imblearn.pipeline.Pipeline) with one of its [oversamplers](https://imbalanced-learn.org/stable/references/over_sampling.html)

</div>

This code trains two different Random Forest Classifier models: one with class weights and the other without upsampling. Both models are evaluated on the same test set for comparison. This approach ensures that the two techniques are applied separately and their performance can be compared directly on the test set.


In [14]:

# Define and train the three logistic regression models
logistic_regression_class_weight = LogisticRegression(random_state=42, class_weight='balanced')
logistic_regression_upsampled = LogisticRegression(random_state=42)
logistic_regression_original = LogisticRegression(random_state=42)  

logistic_regression_class_weight.fit(X_train, y_train)
logistic_regression_upsampled.fit(X_train_upsampled, y_train_upsampled)
logistic_regression_original.fit(X_train, y_train)

# Predict on the validation set and calculate F1 scores and AUC-ROC scores
y_pred_logistic_class_weight = logistic_regression_class_weight.predict(X_temp)
f1_logistic_class_weight = f1_score(y_temp, y_pred_logistic_class_weight)
roc_auc_logistic_class_weight = roc_auc_score(y_temp, logistic_regression_class_weight.predict_proba(X_temp)[:, 1])

y_pred_logistic_upsampled = logistic_regression_upsampled.predict(X_temp)
f1_logistic_upsampled = f1_score(y_temp, y_pred_logistic_upsampled)
roc_auc_logistic_upsampled = roc_auc_score(y_temp, logistic_regression_upsampled.predict_proba(X_temp)[:, 1])

y_pred_logistic_original = logistic_regression_original.predict(X_temp)
f1_logistic_original = f1_score(y_temp, y_pred_logistic_original)
roc_auc_logistic_original = roc_auc_score(y_temp, logistic_regression_original.predict_proba(X_temp)[:, 1])

# Print the F1 scores and AUC-ROC scores for each model on the validation set
print("Logistic Regression with Class Weights (Validation Set):")
print(f"F1 Score: {f1_logistic_class_weight}")
print(f"AUC-ROC Score: {roc_auc_logistic_class_weight}")

print("\nLogistic Regression with Upsampling (Validation Set):")
print(f"F1 Score: {f1_logistic_upsampled}")
print(f"AUC-ROC Score: {roc_auc_logistic_upsampled}")

print("\nLogistic Regression Original (No Balancing) (Validation Set):")
print(f"F1 Score: {f1_logistic_original}")
print(f"AUC-ROC Score: {roc_auc_logistic_original}")

Logistic Regression with Class Weights (Validation Set):
F1 Score: 0.41044776119402987
AUC-ROC Score: 0.7030108919985485

Logistic Regression with Upsampling (Validation Set):
F1 Score: 0.3473861720067454
AUC-ROC Score: 0.5809826612537422

Logistic Regression Original (No Balancing) (Validation Set):
F1 Score: 0.15211267605633802
AUC-ROC Score: 0.6664956227887145


In [16]:
# Evaluate the final model on the test set
y_pred_rf_class_weight_test = best_rf_classifier_class_weight.predict(X_test)
f1_rf_class_weight_test = f1_score(y_test, y_pred_rf_class_weight_test)
roc_auc_rf_class_weight_test = roc_auc_score(y_test, best_rf_classifier_class_weight.predict_proba(X_test)[:, 1])

print("Random Forest Classifier with Class Weights (Test Set):")
print(f"F1 Score: {f1_rf_class_weight_test}")
print(f"AUC-ROC Score: {roc_auc_rf_class_weight_test}")


Random Forest Classifier with Class Weights (Test Set):
F1 Score: 0.6184012066365008
AUC-ROC Score: 0.8624055555555556


<div class="alert alert-danger">
<S><b>Reviewer's comment</b>

You say
    
> Compare F1 scores and AUC-ROC values of the models on the validation set
    
but the models are evaluated on the test set...

</div>

<div class="alert alert-info">
  Corrected
</div>

<div class="alert alert-danger">
<s>
    <b>Reviewer's comment V2</b>

As you already split the data into train, validation and test sets earlier, there's no need to do it again:
    
```python
# Split the data into training (80%) and validation (20%) sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
```
    
Also it seems you're a bit confused about the purpose of these sets. Let's recap:
    
1. The train set is used to train the model
2. The validation set is used to evaluate different models (or variations of the same model with different hyperparameters), compare their results and select the best model. The validation set may be replaced with cross-validation on the train set.
3. The test used is used to evaluate a single final model (not each model)

</div>

<div class="alert alert-success">
<b>Reviewer's comment V3</b>

All looks good now! Well done!

</div>

# Conclusion: Customer Churn Prediction for Beta Bank



In this project, we set out to address Beta Bank's pressing issue of customer churn by developing a machine learning model capable of predicting which customers are likely to leave the bank. By achieving this, Beta Bank can proactively take measures to retain existing customers and reduce the need for acquiring new ones. The project unfolded through several key stages:

Data Preparation: We began by downloading and preprocessing the dataset, ensuring that it was ready for model training. This involved handling missing values, encoding categorical features, and scaling numeric attributes.

Class Imbalance Analysis: We explored the class distribution in the dataset, and it became evident that there was a significant class imbalance with more customers who stayed with the bank than those who left.

Model Training without Balancing: We initially trained machine learning models without addressing the class imbalance. This provided us with baseline performance metrics to assess the severity of the problem.

Improving Model Quality: To combat class imbalance, we explored various techniques, including upsampling the minority class and assigning class weights. Through hyperparameter tuning and grid search, we optimized our models for better predictive accuracy.

Model Training and Validation: We trained multiple models, including Random Forest and Logistic Regression, comparing their performance using F1 scores and AUC-ROC values on validation sets.

Selecting the Best Model: Based on our evaluation, we selected the model with the highest F1 score as the best-performing model. This model demonstrated a strong ability to identify customers at risk of churning.

Final Testing: We assessed the chosen model on a test dataset to obtain its final performance metrics. The model's F1 score and AUC-ROC score on the test set provided a robust evaluation of its predictive capabilities.

By successfully completing this project, we have equipped Beta Bank with a powerful tool to predict customer churn accurately. This predictive capability will allow Beta Bank to take proactive actions, such as personalized retention strategies and improved customer service, to retain valued customers. Ultimately, this approach will not only enhance customer satisfaction but also contribute to cost savings and the long-term success of Beta Bank in a competitive financial landscape.


<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Nice summary!

</div>